In [3]:
%pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [1]:
# models/resnet18.py
import torchvision.models as models
import torch.nn as nn

def get_model(num_classes):
    model = models.resnet18(pretrained=True)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model


In [4]:
# scripts/train_model.py
from sklearn.metrics import f1_score, confusion_matrix
from torchmetrics.classification import F1Score

def evaluate_model(model, dataloader, labels):
    y_true, y_pred = [], []
    for images, targets in dataloader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(targets.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

    print("Custom F1:", f1_score(y_true, y_pred, average='macro'))
    tm_f1 = F1Score(num_classes=len(set(labels)), average='macro')
    print("Torchmetrics F1:", tm_f1(torch.tensor(y_pred), torch.tensor(y_true)))

    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(set(labels)))))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels)
    plt.title("Confusion Matrix")
    plt.savefig("outputs/confusion_matrix.png")


In [5]:
def plot_predictions(correct, incorrect, save_dir="outputs/predictions/"):
    os.makedirs(save_dir, exist_ok=True)
    for i, (img, true, pred) in enumerate(correct[:5]):
        plt.imshow(img.permute(1, 2, 0).numpy())
        plt.title(f"Correct: {true}")
        plt.savefig(f"{save_dir}/correct_{i}.png")

    for i, (img, true, pred) in enumerate(incorrect[:5]):
        plt.imshow(img.permute(1, 2, 0).numpy())
        plt.title(f"Incorrect: True={true}, Pred={pred}")
        plt.savefig(f"{save_dir}/incorrect_{i}.png")
